In [0]:
from delta.tables import DeltaTable

def upsert_into_delta_table(upd_df, table_path,merge_condition,partition_col = None):
    spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning","true")

    if spark.catalog.tableExists(table_path):

        resultsDeltaTable = DeltaTable.forName(spark, table_path)

        resultsDeltaTable.alias('tgt') \
        .merge(
                upd_df.alias('upd'), 
                merge_condition) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
    else:
        if partition_col:
            writer = upd_df.write \
                .format("delta") \
                .mode("overwrite") \
                .partitionBy(partition_col)
        else:
            writer = upd_df.write \
                .format("delta") \
                .mode("overwrite")

        writer.saveAsTable(table_path)

In [0]:
def push_partition_col_to_end(df, partition_col):
    cols = df.schema.names
    cols.remove(partition_col)
    cols.append(partition_col)
    return df.select(cols)